In [ ]:
# Installation des packages nécessaires
install.packages("tidyverse")  # Pour la manipulation et visualisation
install.packages("ggplot2")    # Pour les graphiques
install.packages("dplyr")      # Pour la manipulation de données
install.packages("corrplot")   # Pour les matrices de corrélation
install.packages("psych")      # Pour les statistiques descriptives
install.packages("car")        # Pour les tests statistiques avancés

# Chargement des libraries
library(tidyverse)
library(ggplot2)
library(dplyr)
library(corrplot)
library(psych)
library(car)


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘mnormt’, ‘GPArotation’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘colorspace’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘cowplot’, ‘Deriv’, ‘forecast’, ‘microbenchmark’, ‘rbibutils’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘Rdpack’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’


── Attaching core tidyverse packages ──────────────

In [ ]:
# Charger le dataset
data <- read.csv("Effect_Climate_Change.csv", header = TRUE, stringsAsFactors = TRUE)

# Afficher les premières lignes
head(data)

# Dimensions du dataset
dim(data)
# Structure des données
str(data)

# Résumé statistique
summary(data)

# Vérifier les valeurs manquantes
colSums(is.na(data))

Warning message in file(file, "rt"):
“cannot open file 'Effect_Climate_Change.csv': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


NETTOYAGE DES DONNÉES


In [ ]:

# Supprimer la colonne SNO (numéro de série)
data_clean <- data %>% select(-SNO)

# Renommer les colonnes pour faciliter l'utilisation
names(data_clean) <- make.names(names(data_clean))

# Vérifier les variables catégorielles
categorical_vars <- c("REGION.", "SEX", "MARITAL.STATUS", "EDUCATION",
                     "OCCUPATION", "BMI.CATEGORY", "DISEASE", "DISEASE.TYPE",
                     "FLOODS", "CONTAMINATION.OF.WATER", "COLD.WAVES",
                     "WINTER.STORMS", "CITY")

In [ ]:
# Convertir en facteurs
for(var in categorical_vars) {
  if(var %in% names(data_clean)) {
    data_clean[[var]] <- as.factor(data_clean[[var]])
  }
}


4. STATISTIQUES DESCRIPTIVES

In [ ]:

# Statistiques pour variables numériques
numeric_vars <- c("AGE", "WEIGHT", "HEIGHT", "BMI", "HEAT.WAVES",
                 "DEGREE.OF.DISEASE")

# Créer un tableau de statistiques descriptives
desc_stats <- describe(data_clean[, numeric_vars])
print(desc_stats)

# Tableau de fréquences pour variables catégorielles
cat("\n=== Distribution par REGION ===\n")
table(data_clean$REGION.)

cat("\n=== Distribution par SEXE ===\n")
table(data_clean$SEX)

cat("\n=== Présence de MALADIES ===\n")
table(data_clean$DISEASE)

cat("\n=== Exposition aux INONDATIONS ===\n")
table(data_clean$FLOODS)


5. ANALYSES DE CORRÉLATION

In [ ]:
# Sélectionner les variables numériques pour la corrélation
numeric_data <- data_clean %>%
  select(AGE, WEIGHT, HEIGHT, BMI, HEAT.WAVES, DEGREE.OF.DISEASE,
         FLOODS.descrete, DISEASE.descrete2, CONTAMINATION.OF.WATER.descrete)

# Calculer la matrice de corrélation
cor_matrix <- cor(numeric_data, use = "complete.obs")
print(cor_matrix)

# Visualisation de la matrice de corrélation
corrplot(cor_matrix, method = "color", type = "upper",
         tl.col = "black", tl.srt = 45,
         title = "Matrice de Corrélation - Variables Climatiques et Santé")

# Test de corrélation spécifique : Vagues de chaleur vs Maladies
cor.test(data_clean$HEAT.WAVES, data_clean$DEGREE.OF.DISEASE)

6. VISUALISATIONS


In [ ]:
# 6.1 Distribution de l'âge par région
ggplot(data_clean, aes(x = REGION., y = AGE, fill = REGION.)) +
  geom_boxplot() +
  labs(title = "Distribution de l'Âge par Région",
       x = "Région", y = "Âge") +
  theme_minimal()



In [ ]:
# 6.2 Relation entre BMI et maladies
ggplot(data_clean, aes(x = BMI.CATEGORY, fill = DISEASE)) +
  geom_bar(position = "fill") +
  labs(title = "Proportion de Maladies selon la Catégorie BMI",
       x = "Catégorie BMI", y = "Proportion") +
  theme_minimal() +
  theme(axis.text.x = element_text(angle = 45, hjust = 1))


In [ ]:

# 6.3 Vagues de chaleur par ville
ggplot(data_clean, aes(x = CITY, y = HEAT.WAVES, fill = CITY)) +
  geom_violin() +
  geom_boxplot(width = 0.1, fill = "white", alpha = 0.5) +
  labs(title = "Distribution des Vagues de Chaleur par Ville",
       x = "Ville", y = "Intensité des Vagues de Chaleur") +
  theme_minimal()


In [ ]:

# 6.4 Relation entre contamination de l'eau et maladies
ggplot(data_clean, aes(x = CONTAMINATION.OF.WATER, fill = DISEASE)) +
  geom_bar(position = "dodge") +
  labs(title = "Contamination de l'Eau et Présence de Maladies",
       x = "Contamination de l'Eau", y = "Nombre de Cas") +
  theme_minimal()



In [ ]:
# 6.5 Scatter plot : Vagues de chaleur vs BMI coloré par maladie
ggplot(data_clean, aes(x = HEAT.WAVES, y = BMI, color = DISEASE)) +
  geom_point(alpha = 0.6) +
  geom_smooth(method = "lm", se = TRUE) +
  labs(title = "Relation entre Vagues de Chaleur, BMI et Maladies",
       x = "Intensité des Vagues de Chaleur",
       y = "Indice de Masse Corporelle (BMI)") +
  theme_minimal()

7. TESTS STATISTIQUES


In [ ]:
# 7.1 Test du Chi-carré : Association entre FLOODS et DISEASE
table_floods_disease <- table(data_clean$FLOODS, data_clean$DISEASE)
chi_test_1 <- chisq.test(table_floods_disease)
print("Test Chi-carré : Inondations vs Maladies")
print(chi_test_1)

In [ ]:
# 7.2 Test du Chi-carré : Contamination de l'eau et Maladies
table_water_disease <- table(data_clean$CONTAMINATION.OF.WATER, data_clean$DISEASE)
chi_test_2 <- chisq.test(table_water_disease)
print("Test Chi-carré : Contamination de l'Eau vs Maladies")
print(chi_test_2)


In [ ]:
# 7.3 Test t : Différence de BMI entre ceux exposés aux inondations ou non
t_test_floods <- t.test(BMI ~ FLOODS, data = data_clean)
print("Test t : BMI selon l'exposition aux Inondations")
print(t_test_floods)

In [ ]:
# 7.4 ANOVA : Effet de la ville sur les vagues de chaleur
anova_city <- aov(HEAT.WAVES ~ CITY, data = data_clean)
summary(anova_city)

In [ ]:
# Test post-hoc de Tukey pour comparaisons multiples
tukey_test <- TukeyHSD(anova_city)
print("Test de Tukey : Comparaisons multiples des Villes")
print(tukey_test)


In [ ]:

# 7.5 ANOVA : Effet de la catégorie BMI sur le degré de maladie
data_disease_only <- data_clean %>% filter(DISEASE == "Yes")
anova_bmi <- aov(DEGREE.OF.DISEASE ~ BMI.CATEGORY, data = data_disease_only)
summary(anova_bmi)

8. MODÈLES DE RÉGRESSION


In [ ]:
# 8.1 Régression linéaire simple : Prédire le degré de maladie
model_1 <- lm(DEGREE.OF.DISEASE ~ HEAT.WAVES, data = data_disease_only)
summary(model_1)


In [ ]:
# Visualisation de la régression
ggplot(data_disease_only, aes(x = HEAT.WAVES, y = DEGREE.OF.DISEASE)) +
  geom_point(alpha = 0.5) +
  geom_smooth(method = "lm", color = "red", se = TRUE) +
  labs(title = "Régression : Vagues de Chaleur et Degré de Maladie",
       x = "Intensité des Vagues de Chaleur",
       y = "Degré de Maladie") +
  theme_minimal()


In [ ]:
# 8.2 Régression linéaire multiple
model_2 <- lm(DEGREE.OF.DISEASE ~ HEAT.WAVES + BMI + AGE,
              data = data_disease_only)
summary(model_2)

# Vérification des diagnostics du modèle
par(mfrow = c(2, 2))
plot(model_2)

# Test de multicolinéarité
vif(model_2)

# 8.3 Régression logistique : Prédire la présence de maladie
# Convertir DISEASE en variable binaire
data_clean$DISEASE_binary <- ifelse(data_clean$DISEASE == "Yes", 1, 0)

model_logistic <- glm(DISEASE_binary ~ HEAT.WAVES + BMI + AGE +
                      FLOODS.descrete + CONTAMINATION.OF.WATER.descrete,
                      data = data_clean, family = binomial)
summary(model_logistic)

# Odds ratios
exp(coef(model_logistic))
